Game design E20 with abilities

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JuMP
using HiGHS

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [61]:
ability_add1 = 100

ability_slow_cancel = 200

ability_your_H_their_H = 400
ability_your_H_their_F = 401
ability_your_H_their_L = 402
ability_your_H_their_P = 403
ability_your_H_their_R = 404
ability_your_H_their_S = 405

ability_your_F_their_H = 406
ability_your_F_their_F = 407
ability_your_F_their_L = 408
ability_your_F_their_P = 409
ability_your_F_their_R = 410
ability_your_F_their_S = 411

ability_your_L_their_H = 412
ability_your_L_their_F = 413
ability_your_L_their_L = 414
ability_your_L_their_P = 415
ability_your_L_their_R = 416
ability_your_L_their_S = 417

ability_your_P_their_H = 418
ability_your_P_their_F = 419
ability_your_P_their_L = 420
ability_your_P_their_P = 421
ability_your_P_their_R = 424
ability_your_P_their_S = 425

ability_your_R_their_H = 426
ability_your_R_their_F = 427
ability_your_R_their_L = 428
ability_your_R_their_P = 429
ability_your_R_their_R = 430
ability_your_R_their_S = 431

ability_your_S_their_H = 432
ability_your_S_their_F = 433
ability_your_S_their_L = 434
ability_your_S_their_P = 435
ability_your_S_their_R = 436
ability_your_S_their_S = 437

ability_med_cancel = 600

ability_ninja = 602
ability_eloquent = 610

ability_fast_cancel = 900
ability_hunter = 910
ability_none = 999


ability_name = Dict(
    ability_add1 =>           "student:::::::::",
    ability_your_H_their_H => "::::::::::(H+=H)",
    ability_your_H_their_F => "::::::::::(H+=F)",
    ability_your_H_their_L => "::::::::::(H+=L)",
    ability_your_H_their_P => "scammer:::(H+=P)",
    ability_your_H_their_R => "::::::::::(H+=R)",
    ability_your_H_their_S => "sommelier:(H+=S)",
    ability_your_F_their_H => "::::::::::(F+=H)",
    ability_your_F_their_F => "::::::::::(F+=F)",
    ability_your_F_their_L => "golddigger(F+=L)",
    ability_your_F_their_P => "::::::::::(F+=P)",
    ability_your_F_their_R => "::::::::::(F+=R)",
    ability_your_F_their_S => "coach:::::(F+=S)",
    ability_your_L_their_H => "::::::::::(L+=H)",
    ability_your_L_their_F => "::::::::::(L+=F)",
    ability_your_L_their_L => "::::::::::(L+=L)",
    ability_your_L_their_P => "::::::::::(L+=P)",
    ability_your_L_their_R => "::::::::::(L+=R)",
    ability_your_L_their_S => "hero::::::(L+=S)",
    ability_your_P_their_H => "::::::::::(P+=H)",
    ability_your_P_their_F => "::::::::::(P+=F)",
    ability_your_P_their_L => "paparazzi:(P+=L)",
    ability_your_P_their_P => "::::::::::(P+=P)",
    ability_your_P_their_R => "liontamer:(P+=R)",
    ability_your_P_their_S => "::::::::::(P+=S)",
    ability_your_R_their_H => "::::::::::(R+=H)",
    ability_your_R_their_F => "::::::::::(R+=F)",
    ability_your_R_their_L => "::::::::::(R+=L)",
    ability_your_R_their_P => "::::::::::(R+=P)",
    ability_your_R_their_R => "::::::::::(R+=R)",
    ability_your_R_their_S => "pyro::::::(R+=S)",
    ability_your_S_their_H => "::::::::::(S+=H)",
    ability_your_S_their_F => "::::::::::(S+=F)",
    ability_your_S_their_L => "::::::::::(S+=L)",
    ability_your_S_their_P => "::::::::::(S+=P)",
    ability_your_S_their_R => "frightful:(S+=R)",
    ability_your_S_their_S => "::::::::::(S+=S)",
    ability_ninja =>          "ninja:::::::::::",
    ability_none =>           "vanilla:::::::::",
    ability_eloquent =>       "eloquent::::::::",
    ability_slow_cancel =>    "slowCancel::::::",
    ability_med_cancel =>     "medCancel:::::::",
    ability_fast_cancel =>    "fastCancel::::::",
    ability_hunter =>         "hunter::::::::::",
)

function fx_add1(a::Spud, b::Spud)
    b0 = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.s, ability_none)
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+1, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+1, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p, a.r+1, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+1, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p+1, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p+1, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l+1, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l+1, a.p, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f+1, a.l, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f+1, a.l, a.p, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h+1, a.f, a.l, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h+1, a.f, a.l, a.p, a.r, a.s, ability_none)
    end
    return [a, b]
end

function fx_your_H_their_H(a::Spud, b::Spud)
    a = Spud(a.name, a.h+b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_H_their_F(a::Spud, b::Spud)
    a = Spud(a.name, a.h+b.f, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_H_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h+b.l, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_H_their_P(a::Spud, b::Spud)
    a = Spud(a.name, a.h+b.p, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_H_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h+b.r, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_H_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h+b.s, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_H(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f+b.h, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_F(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f+b.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f+b.l, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_P(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f+b.p, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f+b.r, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f+b.s, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_H(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l+b.h, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_F(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l+b.f, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l+b.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_P(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l+b.p, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l+b.r, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l+b.s, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_H(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p+b.h, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_F(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p+b.f, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p+b.l, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_P(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p+b.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p+b.r, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p+b.s, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_H(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+b.h, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_F(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+b.f, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+b.l, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_P(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+b.p, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+b.r, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+b.s, a.s, a.a1)
    return [a, b]
end

function fx_your_S_their_H(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+b.h, a.a1)
    return [a, b]
end

function fx_your_S_their_F(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+b.f, a.a1)
    return [a, b]
end

function fx_your_S_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+b.l, a.a1)
    return [a, b]
end

function fx_your_S_their_P(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+b.p, a.a1)
    return [a, b]
end

function fx_your_S_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+b.r, a.a1)
    return [a, b]
end

function fx_your_S_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+b.s, a.a1)
    return [a, b]
end

function fx_ninja(a::Spud, b::Spud)
    if (a.h >= b.f) && (a.f >= b.h)
        a = Spud(a.name, a.h, a.f, a.l+1, a.p+1, a.r+1, a.s+1, a.a1)
    end
    return [a, b]
end

function fx_eloquent(a::Spud, b::Spud)
    if (a.p > b.p) && (a.r > b.r) && (a.s > b.s)
        a = Spud(a.name, a.h, a.f, a.l+6, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_cancel(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.s, ability_none)    
    return [a, b]
end

function fx_hunter(a::Spud, b::Spud)
    if (b.p == 1) && (b.r == 1) && (b.s == 1)
        a = Spud(a.name, a.h, a.f+6, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end


ability_fx = Dict(
    ability_ninja => fx_ninja,
    ability_add1 => fx_add1,
    ability_your_H_their_H => fx_your_H_their_H,
    ability_your_H_their_F => fx_your_H_their_F,
    ability_your_H_their_L => fx_your_H_their_L,
    ability_your_H_their_P => fx_your_H_their_P,
    ability_your_H_their_R => fx_your_H_their_R,
    ability_your_H_their_S => fx_your_H_their_S,
    ability_your_F_their_H => fx_your_F_their_H,
    ability_your_F_their_F => fx_your_F_their_F,
    ability_your_F_their_L => fx_your_F_their_L,
    ability_your_F_their_P => fx_your_F_their_P,
    ability_your_F_their_R => fx_your_F_their_R,
    ability_your_F_their_S => fx_your_F_their_S,
    ability_your_L_their_H => fx_your_L_their_H,
    ability_your_L_their_F => fx_your_L_their_F,
    ability_your_L_their_L => fx_your_L_their_L,
    ability_your_L_their_P => fx_your_L_their_P,
    ability_your_L_their_R => fx_your_L_their_R,
    ability_your_L_their_S => fx_your_L_their_S,
    ability_your_P_their_H => fx_your_P_their_H,
    ability_your_P_their_F => fx_your_P_their_F,
    ability_your_P_their_L => fx_your_P_their_L,
    ability_your_P_their_P => fx_your_P_their_P,
    ability_your_P_their_R => fx_your_P_their_R,
    ability_your_P_their_S => fx_your_P_their_S,
    ability_your_R_their_H => fx_your_R_their_H,
    ability_your_R_their_F => fx_your_R_their_F,
    ability_your_R_their_L => fx_your_R_their_L,
    ability_your_R_their_P => fx_your_R_their_P,
    ability_your_R_their_R => fx_your_R_their_R,
    ability_your_R_their_S => fx_your_R_their_S,
    ability_your_S_their_H => fx_your_S_their_H,
    ability_your_S_their_F => fx_your_S_their_F,
    ability_your_S_their_L => fx_your_S_their_L,
    ability_your_S_their_P => fx_your_S_their_P,
    ability_your_S_their_R => fx_your_S_their_R,
    ability_your_S_their_S => fx_your_S_their_S,
    ability_eloquent => fx_eloquent,
    ability_slow_cancel => fx_cancel,
    ability_med_cancel => fx_cancel,
    ability_fast_cancel => fx_cancel,
    ability_hunter => fx_hunter,
)

Dict{Int64, Function} with 43 entries:
  402 => fx_your_H_their_L
  413 => fx_your_L_their_F
  425 => fx_your_P_their_S
  429 => fx_your_R_their_P
  419 => fx_your_P_their_F
  910 => fx_hunter
  418 => fx_your_P_their_H
  408 => fx_your_F_their_L
  420 => fx_your_P_their_L
  404 => fx_your_H_their_R
  417 => fx_your_L_their_S
  405 => fx_your_H_their_S
  431 => fx_your_R_their_S
  437 => fx_your_S_their_S
  430 => fx_your_R_their_R
  434 => fx_your_S_their_L
  410 => fx_your_F_their_R
  406 => fx_your_F_their_H
  411 => fx_your_F_their_S
  414 => fx_your_L_their_L
  100 => fx_add1
  200 => fx_cancel
  415 => fx_your_L_their_P
  409 => fx_your_F_their_P
  412 => fx_your_L_their_H
  ⋮   => ⋮

In [62]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [63]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [64]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [65]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get dishes with cost 20 and abilities

In [66]:
function get_library(cost, ab=999, n_init = 20000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff_i += 1
                                ff = Spud(string(ability_name[ab], "-", h,f,l,p,r,s), h,f,l,p,r,s,ab)
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 3 methods)

In [67]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [68]:
function get_nash_env(lib::Array{Spud})::Tuple{Array{Spud}, Array{Float64}}
    ## optimize
    payoffs = get_payoffs(lib);
    vector_model = Model(HiGHS.Optimizer)
    n_nash = length(lib)
    sumv = zeros(Int64, (1, 2 * n_nash + 1))
    sumv[1:n_nash] .= 1
    A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
    A = vcat(A, sumv);
    b = zeros(Int64, n_nash + 1);
    b[1:n_nash] .= -1
    b[n_nash+1] = 1
    transpose(b);
    c = zeros(Int64, 2 * n_nash + 1);
    c[2 * n_nash + 1] = 1;
    @variable(vector_model, x[1:(2*n_nash+1)] >= 0);
    @constraint(vector_model, A * x .== b);
    @objective(vector_model, Min, c' * x)
    @time optimize!(vector_model)
    ## solution
    w_lp = [value(x[i]) for i in 1:n_nash]
    w_lp = w_lp./sum(w_lp);
    new_env = lib[w_lp .> 0]
    w_new = w_lp[w_lp .> 0];
    for i in sortperm(-w_new)
        print(new_env[i])
        print(" ")
        println(string(w_new[i])[1:6])
    end
    return new_env, w_new
end

get_nash_env (generic function with 1 method)

In [92]:
lib = vcat(
    get_library(20, ability_none),
    get_library(19, ability_ninja),
    get_library(19, ability_hunter),
    get_library(19, ability_slow_cancel),
    get_library(18, ability_med_cancel),
    get_library(18, ability_eloquent),
    get_library(16, ability_add1),
    get_library(16, ability_fast_cancel),
    #get_library(15, ability_your_F_their_H),
    get_library(16, ability_your_F_their_F),
    get_library(16, ability_your_F_their_L),
    get_library(16, ability_your_F_their_P),
    get_library(16, ability_your_F_their_R),
    get_library(18, ability_your_F_their_S),
    get_library(16, ability_your_H_their_H),
    #get_library(15, ability_your_H_their_F),
    get_library(16, ability_your_H_their_L),
    get_library(17, ability_your_H_their_P),
    get_library(16, ability_your_H_their_R),
    get_library(17, ability_your_H_their_S),
    get_library(16, ability_your_L_their_H),
    get_library(16, ability_your_L_their_F),
    #get_library(15, ability_your_L_their_L),
    get_library(16, ability_your_L_their_P),
    get_library(16, ability_your_L_their_R),
    get_library(16, ability_your_L_their_S),
    get_library(16, ability_your_P_their_H),
    get_library(16, ability_your_P_their_F),
    get_library(18, ability_your_P_their_L),
    get_library(15, ability_your_P_their_P),
    get_library(17, ability_your_P_their_R),
    get_library(16, ability_your_P_their_S),
    get_library(16, ability_your_R_their_H),
    get_library(16, ability_your_R_their_F),
    get_library(16, ability_your_R_their_L),
    get_library(16, ability_your_R_their_P),
    get_library(15, ability_your_R_their_R),
    get_library(17, ability_your_R_their_S),
    get_library(16, ability_your_S_their_H),
    get_library(16, ability_your_S_their_F),
    get_library(16, ability_your_S_their_L),
    get_library(16, ability_your_S_their_P),
    get_library(17, ability_your_S_their_R),
    get_library(15, ability_your_S_their_S),
)
length(lib)

158333

In [93]:
sublib = unique([rand(lib) for i in 1:3000])
@time ne, wn = get_nash_env(sublib);

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2976 rows, 5951 cols, 8856323 nonzeros
2976 rows, 5951 cols, 8856323 nonzeros
Presolve : Reductions: rows 2976(-0); columns 5951(-0); elements 8856323(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2976(2976) 1s
       3503     9.1536687254e-01 Pr: 1543(6668.78) 6s
       4197     1.0000000000e+00 Pr: 0(0) 7s
       4197     1.0000000000e+00 Pr: 0(0) 7s
Model   status      : Optimal
Simplex   iterations: 4197
Objective value     :  1.0000000000e+00
HiGHS run time      :          7.86
  8.249548 seconds (50.83 k allocations: 510.984 MiB, 0.35% gc time)
Spud("slowCancel::::::-915211", 9, 1, 5, 2, 1, 1, 200) 0.0789
Spud("vanilla:::::::::-161264", 1, 6, 1, 2, 6, 4, 999) 0.0423
Spud("::::::::::(H+=R)-216142", 2, 1, 6, 1, 4, 2, 404) 0.0419
Spud("::::::::::

In [94]:
@time scores = [eval_battle_list2(ff, ne, wn) for ff in lib];
sublib = lib[sortperm(-scores)[1:2000]]
@time ne, wn = get_nash_env(sublib);

 13.803258 seconds (280.35 M allocations: 23.404 GiB, 10.62% gc time, 0.36% compilation time)
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2001 rows, 4001 cols, 4002796 nonzeros
2001 rows, 4001 cols, 4002796 nonzeros
Presolve : Reductions: rows 2001(-0); columns 4001(-0); elements 4002796(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2001(2001) 0s
       3409     1.0000000000e+00 Pr: 0(0) 4s
       3409     1.0000000000e+00 Pr: 0(0) 4s
Model   status      : Optimal
Simplex   iterations: 3409
Objective value     :  1.0000000000e+00
HiGHS run time      :          4.12
  4.280855 seconds (32.24 k allocations: 232.165 MiB)
Spud("::::::::::(L+=H)-813211", 8, 1, 3, 2, 1, 1, 412) 0.0267
Spud("vanilla:::::::::-131177", 1, 3, 1, 1, 7, 7, 999) 0.0261
Spud("medCancel:::::::-319221", 3, 1, 9, 2, 2, 1, 600

In [95]:
@time scores = [eval_battle_list2(ff, ne, wn) for ff in lib];
sublib = lib[sortperm(-scores)[1:2000]]
@time ne, wn = get_nash_env(sublib);

 29.528354 seconds (663.09 M allocations: 54.416 GiB, 10.93% gc time, 0.05% compilation time)
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2001 rows, 4001 cols, 4003658 nonzeros
2001 rows, 4001 cols, 4003658 nonzeros
Presolve : Reductions: rows 2001(-0); columns 4001(-0); elements 4003658(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2001(2001) 0s
       2929     9.7224134197e-01 Pr: 1089(615.571) 5s
       4649     1.0000000000e+00 Pr: 0(0) 8s
       4649     1.0000000000e+00 Pr: 0(0) 8s
Model   status      : Optimal
Simplex   iterations: 4649
Objective value     :  1.0000000000e+00
HiGHS run time      :          8.77
  8.930988 seconds (32.24 k allocations: 232.274 MiB)
Spud("vanilla:::::::::-917111", 9, 1, 7, 1, 1, 1, 999) 0.0297
Spud("hero::::::(L+=S)-319111", 3, 1, 9, 1, 1, 1, 417) 0.0275

Spud("vanilla:::::::::-191432", 1, 9, 1, 4, 3, 2, 999) 0.0018
Spud("frightful:(S+=R)-171521", 1, 7, 1, 5, 2, 1, 436) 0.0018
Spud("::::::::::(P+=P)-114171", 1, 1, 4, 1, 7, 1, 421) 0.0018
Spud("frightful:(S+=R)-151712", 1, 5, 1, 7, 1, 2, 436) 0.0018
Spud("fastCancel::::::-615121", 6, 1, 5, 1, 2, 1, 900) 0.0018
Spud("ninja:::::::::::-942112", 9, 4, 2, 1, 1, 2, 602) 0.0017
Spud("::::::::::(S+=S)-141531", 1, 4, 1, 5, 3, 1, 437) 0.0016
Spud("::::::::::(R+=R)-115314", 1, 1, 5, 3, 1, 4, 430) 0.0016
Spud("scammer:::(H+=P)-116513", 1, 1, 6, 5, 1, 3, 403) 0.0016
Spud("paparazzi:(P+=L)-171261", 1, 7, 1, 2, 6, 1, 420) 0.0015
Spud("pyro::::::(R+=S)-151712", 1, 5, 1, 7, 1, 2, 431) 0.0015
Spud("student:::::::::-219211", 2, 1, 9, 2, 1, 1, 100) 0.0015
Spud("eloquent::::::::-115236", 1, 1, 5, 2, 3, 6, 610) 0.0015
Spud("medCancel:::::::-114327", 1, 1, 4, 3, 2, 7, 600) 0.0015
Spud("hunter::::::::::-131176", 1, 3, 1, 1, 7, 6, 910) 0.0015
Spud("vanilla:::::::::-181451", 1, 8, 1, 4, 5, 1, 999) 0.0015
Spud("ni

In [96]:
@time scores = [eval_battle_list2(ff, ne, wn) for ff in lib];
sublib = lib[sortperm(-scores)[1:2000]]
@time ne, wn = get_nash_env(sublib);

 44.932236 seconds (1.05 G allocations: 86.320 GiB, 10.95% gc time, 0.06% compilation time)
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2001 rows, 4001 cols, 4003456 nonzeros
2001 rows, 4001 cols, 4003456 nonzeros
Presolve : Reductions: rows 2001(-0); columns 4001(-0); elements 4003456(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2001(2001) 0s
       3022     9.9057449317e-01 Pr: 943(272.642) 5s
       5276     1.0000567792e+00 Pr: 0(0) 11s
       5281     1.0000000000e+00 Pr: 0(0) 11s
       5281     1.0000000000e+00 Pr: 0(0) 11s
Model   status      : Optimal
Simplex   iterations: 5281
Objective value     :  1.0000000000e+00
HiGHS run time      :         11.17
 11.332167 seconds (32.24 k allocations: 232.230 MiB)
Spud("vanilla:::::::::-917111", 9, 1, 7, 1, 1, 1, 999) 0.0209
Spud("hero::::::

Spud("frightful:(S+=R)-151721", 1, 5, 1, 7, 2, 1, 436) 0.0023
Spud("eloquent::::::::-112347", 1, 1, 2, 3, 4, 7, 610) 0.0023
Spud("vanilla:::::::::-219134", 2, 1, 9, 1, 3, 4, 999) 0.0023
Spud("::::::::::(S+=S)-141351", 1, 4, 1, 3, 5, 1, 437) 0.0023
Spud("vanilla:::::::::-141716", 1, 4, 1, 7, 1, 6, 999) 0.0022
Spud("golddigger(F+=L)-131641", 1, 3, 1, 6, 4, 1, 408) 0.0022
Spud("::::::::::(S+=S)-141261", 1, 4, 1, 2, 6, 1, 437) 0.0022
Spud("::::::::::(P+=P)-151125", 1, 5, 1, 1, 2, 5, 421) 0.0022
Spud("ninja:::::::::::-634114", 6, 3, 4, 1, 1, 4, 602) 0.0022
Spud("student:::::::::-219112", 2, 1, 9, 1, 1, 2, 100) 0.0022
Spud("vanilla:::::::::-142715", 1, 4, 2, 7, 1, 5, 999) 0.0022
Spud("medCancel:::::::-131355", 1, 3, 1, 3, 5, 5, 600) 0.0022
Spud("fastCancel::::::-615211", 6, 1, 5, 2, 1, 1, 900) 0.0022
Spud("ninja:::::::::::-191314", 1, 9, 1, 3, 1, 4, 602) 0.0021
Spud("::::::::::(R+=R)-151314", 1, 5, 1, 3, 1, 4, 430) 0.0021
Spud("ninja:::::::::::-619111", 6, 1, 9, 1, 1, 1, 602) 0.0021
Spud("el

Spud("student:::::::::-222145", 2, 2, 2, 1, 4, 5, 100) 0.0002
Spud("scammer:::(H+=P)-517121", 5, 1, 7, 1, 2, 1, 403) 0.0002
Spud("ninja:::::::::::-761311", 7, 6, 1, 3, 1, 1, 602) 0.0002
Spud("medCancel:::::::-617112", 6, 1, 7, 1, 1, 2, 600) 0.0001
Spud("sommelier:(H+=S)-216314", 2, 1, 6, 3, 1, 4, 405) 0.0001
Spud("::::::::::(L+=H)-912121", 9, 1, 2, 1, 2, 1, 412) 0.0001
Spud("golddigger(F+=L)-111166", 1, 1, 1, 1, 6, 6, 408) 0.0001
Spud("sommelier:(H+=S)-118214", 1, 1, 8, 2, 1, 4, 405) 9.7636
Spud("sommelier:(H+=S)-216143", 2, 1, 6, 1, 4, 3, 405) 7.5523
Spud("coach:::::(F+=S)-131166", 1, 3, 1, 1, 6, 6, 411) 7.0748
Spud("vanilla:::::::::-815114", 8, 1, 5, 1, 1, 4, 999) 6.0912
Spud("medCancel:::::::-132516", 1, 3, 2, 5, 1, 6, 600) 3.6073
Spud("eloquent::::::::-114444", 1, 1, 4, 4, 4, 4, 610) 6.8437
Spud("hunter::::::::::-924112", 9, 2, 4, 1, 1, 2, 910) 6.2527
 17.513858 seconds (124.50 M allocations: 11.125 GiB, 3.49% gc time)


In [97]:
@time scores = [eval_battle_list2(ff, ne, wn) for ff in lib];
sublib = lib[sortperm(-scores)[1:2000]];
maximum(scores), sum(scores .> 0)

 67.831805 seconds (1.51 G allocations: 123.981 GiB, 10.65% gc time, 0.03% compilation time)


(0.05484875141701041, 216)

In [98]:
weights = Dict(k => 0.0 for k in keys(ability_name))
for i in 1:length(ne)
    ff = ne[i]
    weights[ff.a1] += wn[i] 
end
for k in keys(weights)
    print(ability_name[k])
    print(" ")
    println(weights[k])
end

::::::::::(H+=L) 0.0
slowCancel:::::: 0.0
::::::::::(L+=F) 0.0
::::::::::(L+=P) 0.0
::::::::::(P+=S) 0.0
::::::::::(R+=P) 0.0
::::::::::(P+=F) 0.0
hunter:::::::::: 0.025050806052242382
::::::::::(P+=H) 0.0
golddigger(F+=L) 0.04159843425895376
::::::::::(F+=P) 0.0
paparazzi:(P+=L) 0.04004450073491788
::::::::::(H+=R) 0.0
::::::::::(L+=H) 0.0447230517727763
fastCancel:::::: 0.01888968223369954
liontamer:(P+=R) 0.006803318532601063
::::::::::(R+=L) 0.0
medCancel::::::: 0.08573969044624472
hero::::::(L+=S) 0.017174998393375187
::::::::::(H+=F) 0.0
sommelier:(H+=S) 0.01581246460264981
pyro::::::(R+=S) 0.0
::::::::::(S+=S) 0.032666040001483015
vanilla::::::::: 0.24534503921874612
eloquent:::::::: 0.14604234465307514
::::::::::(R+=R) 0.04102734171355584
::::::::::(S+=L) 0.0
::::::::::(F+=R) 0.0
frightful:(S+=R) 0.028326483796049493
::::::::::(P+=P) 0.05291764650721509
::::::::::(S+=P) 0.0
::::::::::(F+=F) 0.0
::::::::::(R+=H) 0.0
::::::::::(R+=F) 0.0
::::::::::(S+=F) 0.0
::::::::::(F+=H) 0.0


In [101]:
# df = spuds_to_df(ne)
# df[!, :w] = [floor(w*1000)/1000 for w in wn]
# print(df[sortperm(-wn),:])
for i in sortperm(-wn)
    print(ne[i].name)
    print(" ")
    println(string(wn[i])[1:7])
end

vanilla:::::::::-917111 0.02095
hero::::::(L+=S)-319111 0.01717
::::::::::(L+=H)-813121 0.01489
::::::::::(L+=H)-517111 0.01290
vanilla:::::::::-718211 0.01231
vanilla:::::::::-419321 0.01195
ninja:::::::::::-429112 0.01156
vanilla:::::::::-519131 0.01154
vanilla:::::::::-619211 0.01151
vanilla:::::::::-916121 0.01140
::::::::::(R+=R)-151512 0.01067
eloquent::::::::-113454 0.01014
student:::::::::-319111 0.01014
eloquent::::::::-114543 0.00962
vanilla:::::::::-161165 0.00956
eloquent::::::::-113463 0.00940
medCancel:::::::-219231 0.00920
paparazzi:(P+=L)-181152 0.00870
vanilla:::::::::-171164 0.00858
eloquent::::::::-121365 0.00833
golddigger(F+=L)-121651 0.00827
::::::::::(L+=H)-814111 0.00815
paparazzi:(P+=L)-161262 0.00786
vanilla:::::::::-319313 0.00739
golddigger(F+=L)-713212 0.00738
::::::::::(R+=R)-151215 0.00736
::::::::::(R+=R)-161214 0.00724
ninja:::::::::::-681112 0.00702
vanilla:::::::::-619121 0.00694
eloquent::::::::-113634 0.00687
ninja:::::::::::-519211 0.00687
liontame

hunter::::::::::-924112 6.25274


In [102]:
length(ne)

321

In [30]:
lib0=lib;

In [ ]:
lib = sublib
payoffs = get_payoffs(lib)
nlib = length(lib)
combos = Array{Spud}(undef, (10000,3))
#ord = sortperm([rand() for i in 1:nlib])[1:800]
#ord = [i for i in 1:nlib]
ord = [i for i in 1:600]
nc = 0
# 3-combos
for i in ord[1:(length(ord)-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(length(ord)-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            #print(".")
            for k in ord[(j+1):length(ord)]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = lib[[i,j,k]]
                    #println("")
                    #println("----3-COMBO!-----")
                    #println(lib[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:];

In [ ]:
size(combos)

### combo-breaker research

In [ ]:
for i in 1:20
    ind = rand(1:size(combos)[1])
    as = combos[ind,:]
    print(ind)
    print(" ")
    println(spuds_to_df(as))
end

In [ ]:
as = combos[1,:]
print(spuds_to_df(as))

In [ ]:
println(maximum([eval_battle_list(ff, as) for ff in lib]))
b = Spud("breaker",1,2,4,1,1,4,ability_your_H_their_P)
println([eval_battle(b, ff) for ff in as])
eval_battle_list2(b, ne, wn)